In [3]:
#!pip install python-Levenshtein

In [1]:
import gensim
import pandas as pd

In [2]:
df = pd.read_json("Cell_Phones_and_Accessories.json", lines=True, nrows=194439 )

In [3]:
df.shape

(194439, 12)

In [4]:
df.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,image,style
0,5,False,"11 19, 2014",A1GG51FWU0XQYH,098949232X,Paul Williams,If your into space this is the Calendar for you.,Five Stars,1416355200,NaN,NaN,NaN
1,5,False,"11 19, 2014",AVFIDS9RK38E0,098949232X,Sean Powell,Awesome pictures!,Five Stars,1416355200,NaN,NaN,NaN
2,5,False,"11 19, 2014",A2S4AVR5SJ7KMI,098949232X,Tom Davis,Great wall art and information for space explo...,Five Stars,1416355200,NaN,NaN,NaN
3,5,False,"11 19, 2014",AEMMMVOR9BFLI,098949232X,Kwajmeck,"As always, it is a quality calendar full of ve...",I love it. I buy a new one every year,1416355200,NaN,NaN,NaN
4,5,False,"11 19, 2014",A2DZXMBTY7KLYP,098949232X,ScottG43,This is a fantastic calendar. This is my third...,Great Calendar.,1416355200,NaN,NaN,NaN


In [5]:
df.isnull().sum()

overall                0
verified               0
reviewTime             0
reviewerID             0
asin                   0
reviewerName          11
reviewText            28
summary               24
unixReviewTime         0
vote              159783
image             193575
style             148969
dtype: int64

In [6]:
df.drop(['vote','image','style'], axis = 1, inplace=True)

In [7]:
df.columns

Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'reviewerName', 'reviewText', 'summary', 'unixReviewTime'],
      dtype='object')

In [8]:
df.dropna(subset=['reviewText'],inplace=True)

In [9]:
reviewtext = df.reviewText.apply(gensim.utils.simple_preprocess)
reviewtext

0         [if, your, into, space, this, is, the, calenda...
1                                       [awesome, pictures]
2         [great, wall, art, and, information, for, spac...
3         [as, always, it, is, quality, calendar, full, ...
4         [this, is, fantastic, calendar, this, is, my, ...
                                ...                        
194434    [and, the, price, is, right, just, be, careful...
194435    [if, think, to, fix, ur, screen, this, is, the...
194436    [this, product, was, exactly, what, needed, my...
194437    [so, got, this, and, had, all, the, confidence...
194438    [the, day, gt, the, item, in, it, came, all, b...
Name: reviewText, Length: 194411, dtype: object

In [10]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)

In [11]:
model.build_vocab(reviewtext, progress_per=1000)

In [33]:
model.epochs

5

In [18]:
model.corpus_count

194411

In [19]:
model.train(reviewtext, total_examples=model.corpus_count, epochs=model.epochs)

(48984677, 66298290)

In [38]:
model.save("./Word2Vec_cellPhoneReview.model")

In [20]:
model.wv.most_similar("meticulous")

[('gross', 0.5250231623649597),
 ('relaxing', 0.5210461020469666),
 ('glorious', 0.5208073258399963),
 ('tiniest', 0.51731276512146),
 ('guilty', 0.5165615677833557),
 ('soothing', 0.510646641254425),
 ('cristal', 0.5072171092033386),
 ('sprayed', 0.5065256357192993),
 ('sparkly', 0.4945668876171112),
 ('ish', 0.4944678246974945)]

In [43]:
model.wv.similarity(w1="good", w2="excellent")

0.6703054

In [25]:
model.wv.get_vector("good", norm=True)

array([ 0.09097162,  0.08851554,  0.01132196, -0.11369427, -0.10771075,
       -0.0751113 , -0.06356426, -0.17785248,  0.0881972 ,  0.04092099,
        0.16606693, -0.06294592,  0.0126646 , -0.12442543,  0.04439859,
       -0.10171369,  0.04110705, -0.14625336,  0.03391758, -0.22374538,
        0.11030564,  0.09268478, -0.05844152, -0.08571649,  0.05608038,
        0.17100948, -0.05050996, -0.14422442,  0.07413103, -0.1325263 ,
        0.1245209 ,  0.03308031, -0.14501259, -0.03488262, -0.04455252,
       -0.08885471,  0.16459511,  0.04223958, -0.18182789,  0.08574769,
        0.04447589, -0.02765994,  0.12003502, -0.03773129, -0.0851194 ,
       -0.15158148,  0.01083875,  0.05961479, -0.07157851, -0.0906005 ,
        0.00337408,  0.0941468 ,  0.06383246,  0.17529014,  0.03803365,
        0.08725906, -0.06006895, -0.04390658,  0.06991064, -0.05708609,
       -0.03282199, -0.14792357,  0.12967816, -0.11337789,  0.23706305,
        0.13507584,  0.09988891,  0.01926849,  0.10781994, -0.15

In [26]:
all_normed_vectors = model.wv.get_normed_vectors()

In [33]:
all_normed_vectors.shape

(36606, 100)

In [31]:
len(all_normed_vectors[0])

100

In [ ]:
my_dict = dict({})
for idx, key in enumerate(model.wv.vocab):
    my_dict[key] = model.wv[key]